# 1.Dataset Processing (Model A)
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

The section immideately below contains all necessray imports.

In [ ]:
#import libraries
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import json
import copy

import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


The section immideately below contains token sequence the preprosessing steps

The loading of the Traning, Val, and Testing datasets:

In [ ]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Get the file ID of the JSON file you want to open
test_id = "1PIaL_FR_NKabc-qKIi_RpaRwsoGgaPqw"
train_id = "1o3dgEDYno-d2WYM9w3tFE_8jF8pDYzyU"
val_id = "1TZZO6Pg16dI_flowwryrF0ZKfyBhrHDf"

# Download the file
test = drive.CreateFile({'id': test_id})
test.GetContentFile('test_data.json')
train = drive.CreateFile({'id': train_id})
train.GetContentFile('train_data.json')
val = drive.CreateFile({'id': val_id})
val.GetContentFile('val_data.json')

# Read the downloaded JSON file
with open('test_data.json', 'r') as f:
    test_data = json.load(f)

with open('train_data.json', 'r') as f:
    train_data = json.load(f)

with open('val_data.json', 'r') as f:
    val_data = json.load(f)


# Function to format rows from the JSON data
def format_rows(data):
    formatted_rows = []
    index = 0
    for row in data:
        sentence = row[0]
        aspect = row[1]
        polarity = row[2]
        formatted_rows.append([index, sentence, aspect, polarity])
        index += 1
    return formatted_rows


rows = test_data['data']
test_data_formatted = format_rows(rows)
rows = train_data['data']
train_data_formatted = format_rows(rows)
rows = val_data['data']
val_data_formatted = format_rows(rows)

print(val_data_formatted[0])

[0, 'I would wait for a table next time, the food was that good.', 'miscellaneous', 'neutral']


In [ ]:
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have",
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not",
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did",
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have",
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have",
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us",
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have",
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have",
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not",
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have",
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have",
                    "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will",
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have",
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have",
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}


def pre_process(sent_list):
    output = []
    for sent in sent_list:
        sent = sent.lower() #case-folding
        for word, new_word in contraction_dict.items():
            sent = sent.replace(word, new_word) #dealing with contractions
        sent = re.sub(r'[^\w\s]','',sent) #removing punctuation
        output.append(word_tokenize(sent)) #tokenization
    return output

In [ ]:
# construct token index lists for input, output and target
# i.e., to convert each sequence into corresponding index based on the index dictionary we created in the previous section
# e.g., ['are', 'your', 'systems', 'functioning'] -> [42, 6, 2576, 2577]
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list


def make_batch(data, to_ix):
    #cap = max(len(lst) for lst in data)
    cap = batch_size
    padded = []
    for x in data:
      if len(x) < cap:
        w = []
        for _ in range(cap-len(x)):
          w.append(to_ix["<pad>"])
        padded.append(x + w)
      else:
        padded.append(x)

    padded = torch.tensor(padded)

    return padded.to(device)


def get_batch(ipt, lbl, i, bptt):
    seq_len = min(bptt, len(ipt) - 1 - i)
    data = ipt[i:i+seq_len].to(device)
    target = lbl[i:i+seq_len].to(device)

    if len(data) < bptt:
        pad_len = bptt - len(data)
        pad_data = torch.zeros(pad_len, data.size(1), dtype=data.dtype).fill_(0).to(device)
        data = torch.cat([data, pad_data], dim=0)
        pad_target = torch.zeros(pad_len, target.size(1), dtype=target.dtype).fill_(0).to(device)
        target = torch.cat([target, pad_target], dim=0)

    return data, target

In [ ]:
# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available on this device.")
    # Get the number of CUDA devices
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")

    # Loop through and print each GPU's information
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  Memory Allocated: {torch.cuda.memory_allocated(i)} bytes")
        print(f"  Memory Cached: {torch.cuda.memory_reserved(i)} bytes")
        print(f"  Compute Capability: {torch.cuda.get_device_properties(i).major}.{torch.cuda.get_device_properties(i).minor}")
else:
    print("CUDA is not available. No GPU detected.")

CUDA is available on this device.
Number of GPUs available: 1
GPU 0: Tesla T4
  Memory Allocated: 0 bytes
  Memory Cached: 0 bytes
  Compute Capability: 7.5


In [ ]:
def Preprocessing(Source):
    sentences = []
    aspects = []
    labels = []
    for row in Source:
        sentences.append(row[1])
        aspects.append(row[2])
        labels.append(row[3])

    # Preprocessing the data using the function defined above
    input_token_list = pre_process(sentences)
    label_token_list = pre_process(labels)

    input_token_list_n = []
    for x in range(len(input_token_list)):
      input_token_list_n.append(input_token_list[x] + ["<HL>"] + [aspects[x]])

    #print(input_token_list_n)

    target_token_list = label_token_list

    # set up a vocab to index dictionary
    word_to_ix = {"<pad>": 0, "<HL>": 1}
    for sentence in input_token_list_n:
        for word in sentence:
            if word not in word_to_ix:
                word_to_ix[word] = len(word_to_ix)
    #print(word_to_ix)

    label_to_index = {
        "positive": 0,
        "negative": 1,
        "neutral": 2
    }

    input_index = to_index(input_token_list_n, word_to_ix)
    target_index = to_index(target_token_list, label_to_index)

    input = make_batch(input_index, word_to_ix)
    #print(input.size())
    #print(input[0])

    return input, target_index, word_to_ix

# 2. Model Implementation (Model A)
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

Hyperparameters

In [ ]:
OUTPUT_SIZE = 3  # target vocab size
HIDDEN_SIZE = 512
N_LAYERS = 1
N_HEADS = 8
FF_SIZE = 2048
DROPOUT_RATE = 0.6
CLIP = 1
N_EPOCHS = 50
batch_size = 80
bptt = batch_size
print(batch_size)

80


Scaled Dot Product Attention

In [ ]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, scale, dropout_rate):
        super(ScaledDotProductAttention, self).__init__()
        self.scale = scale
        self.dropout_rate = dropout_rate

        self.dropout = nn.Dropout(dropout_rate)


    def forward(self, query, key, value):
        # calculate alignment scores
        scores = torch.matmul(query, key.transpose(-2, -1))  # (batch_size, n_heads, query_len, value_len)
        scores = scores / self.scale  # (batch_size, num_heads, query_len, value_len)

        # calculate the attention weights (prob) from alignment scores
        attn_probs = F.softmax(scores, dim=-1)  # (batch_size, n_heads, query_len, value_len)

        # calculate context vector
        output = torch.matmul(self.dropout(attn_probs), value)  # (batch_size, n_heads, query_len, head_dim)

        return output, attn_probs

Multi-head Attention

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout_rate=0.1):
        super(MultiHeadAttention, self).__init__()
        assert d_model % n_heads == 0, "`d_model` should be a multiple of `n_heads`"

        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = self.d_v = d_model // n_heads  # head_dim
        self.dropout_rate = dropout_rate

        self.W_q = nn.Linear(d_model, d_model, bias=False)  # linear transformation for the Q, K and V vectors
        self.W_k = nn.Linear(d_model, d_model, bias=False)
        self.W_v = nn.Linear(d_model, d_model, bias=False)
        self.W_o = nn.Linear(d_model, d_model)
        self.attention = ScaledDotProductAttention(np.sqrt(self.d_k), dropout_rate)


    def project(self, x):
        batch_size = x.size(0)
        x = x.view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)  # (batch_size, n_heads, seq_len, d_k)

        return x


    def unproject(self, x):
        batch_size = x.size(0)
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.n_heads * self.d_k)

        return x


    def forward(self, query, key, value):
        # apply linear projections to query, key and value
        Q = self.project(self.W_q(query))  # (batch_size, n_heads, query_len, head_dim)
        K = self.project(self.W_k(key))  # (batch_size, n_heads, key_len, head_dim)
        V = self.project(self.W_v(value))  # (batch_size, n_heads, value_len, head_dim)

        # calculate attention weights and context vector for each of the heads
        x, attn = self.attention(Q, K, V)

        # concatenate context vector of all the heads
        x = self.unproject(x)  # (batch_size, query_len, d_model)

        # apply linear projection to concatenated context vector
        x = self.W_o(x)

        return x, attn

Positional Encoding

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, vocab_size=6000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        encoding = torch.zeros(vocab_size, d_model)
        position = torch.arange(0, vocab_size, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float()
            * (-math.log(10000.0) / d_model)
        )
        encoding[:, 0::2] = torch.sin(position * div_term)
        encoding[:, 1::2] = torch.cos(position * div_term)
        encoding = encoding.unsqueeze(0)
        self.register_buffer("encoding", encoding)


    def forward(self, x):
        x = x + self.encoding[:, : x.size(1), :]
        x = self.dropout(x)
        return x

Position-wise FF Network

In [ ]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout_rate=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff
        self.dropout_rate = dropout_rate

        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout_rate)


    def forward(self, x):
        x = self.dropout(F.relu(self.w_1(x)))  # (batch_size, seq_len, d_ff)
        x = self.w_2(x)  # (batch_size, seq_len, d_model)

        # x: (batch_size, seq_len, d_model)
        return x

Encoder

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout_rate=0.1):
        super(EncoderLayer, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff
        self.n_heads = n_heads
        self.dropout_rate = dropout_rate

        self.attn_layer = MultiHeadAttention(d_model, n_heads, dropout_rate)
        self.attn_layer_norm = nn.LayerNorm(d_model, eps=1e-6)

        self.ff_layer = PositionwiseFeedForward(d_model, d_ff, dropout_rate)
        self.ff_layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout = nn.Dropout(dropout_rate)


    def forward(self, x):
        # apply self-attention
        x1, _ = self.attn_layer(x, x, x)  # (batch_size, source_seq_len, d_model)

        # apply residual connection followed by layer normalization
        x = self.attn_layer_norm(x + self.dropout(x1))  # (batch_size, source_seq_len, d_model)

        # apply position-wise feed-forward
        x1 = self.ff_layer(x)  # (batch_size, source_seq_len, d_model)

        # apply residual connection followed by layer normalization
        x = self.ff_layer_norm(x + self.dropout(x1))  # (batch_size, source_seq_len, d_model)

        # x: (batch_size, source_seq_len, d_model)
        return x

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, n_layers, n_heads, d_ff, pad_idx, dropout_rate=0.1, max_len=5000):
        super(Encoder, self).__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.n_layers = n_layers
        self.d_ff = d_ff
        self.pad_idx = pad_idx
        self.dropout_rate = dropout_rate
        self.max_len = max_len

        self.tok_embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_embedding = PositionalEncoding(d_model, dropout_rate, max_len)
        self.layers = nn.ModuleList([
            EncoderLayer(d_model, n_heads, d_ff, dropout_rate)
            for _ in range(n_layers)
        ])      # designed for multiple layers
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)


    def forward(self, x):
        # apply positional encoding to token sequences
        x = self.tok_embedding(x)  # (batch_size, source_seq_len, d_model)
        x = self.pos_embedding(x)  # (batch_size, source_seq_len, d_model)

        for layer in self.layers:
            x = layer(x)  # (batch_size, source_seq_len, d_model)

        x = self.layer_norm(x)  # (batch_size, source_seq_len, d_model)

        # x: (batch_size, source_seq_len, d_model)
        return x

Decoder

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout_rate=0.1, num_classes=None):
        super(DecoderLayer, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff
        self.n_heads = n_heads
        self.dropout_rate = dropout_rate
        self.attn_layer = MultiHeadAttention(d_model, n_heads, dropout_rate)
        self.attn_layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.enc_attn_layer = MultiHeadAttention(d_model, n_heads, dropout_rate)
        self.enc_attn_layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.ff_layer = PositionwiseFeedForward(d_model, d_ff, dropout_rate)
        self.ff_layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout = nn.Dropout(dropout_rate)

        # Classification layer
        self.num_classes = num_classes
        if self.num_classes is not None:
            self.classifier = nn.Linear(d_model, num_classes)

    def forward(self, x, memory):
        # apply self-attention
        x1, _ = self.attn_layer(x, x, x)
        # (batch_size, target_seq_len, d_model)
        # apply residual connection followed by layer normalization
        x = self.attn_layer_norm(x + self.dropout(x1))
        # (batch_size, target_seq_len, d_model)

        # apply encoder-decoder attention
        # memory is the output from encoder block (encoder states)
        x1, attn = self.enc_attn_layer(x, memory, memory)
        # x1: (batch_size, target_seq_len, d_model)
        # attn: (batch_size, n_heads, target_seq_len, source_seq_len)
        # apply residual connection followed by layer normalization
        x = self.enc_attn_layer_norm(x + self.dropout(x1))
        # (batch_size, target_seq_len, d_model)

        # apply position-wise feed-forward
        x1 = self.ff_layer(x)
        # (batch_size, target_seq_len, d_model)
        # apply residual connection followed by layer normalization
        x = self.ff_layer_norm(x + self.dropout(x1))
        # (batch_size, target_seq_len, d_model)

        # x: (batch_size, target_seq_len, d_model) or (batch_size, num_classes)
        # attn: (batch_size, n_heads, target_seq_len, source_seq_len)
        return x, attn

In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, n_layers, n_heads, d_ff, pad_idx, dropout_rate=0.1, max_len=5000):
        super(Decoder, self).__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.n_layers = n_layers
        self.d_ff = d_ff
        self.pad_idx = pad_idx
        self.dropout_rate = dropout_rate
        self.max_len = max_len

        self.tok_embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_embedding = PositionalEncoding(d_model, dropout_rate, max_len)
        self.layers = nn.ModuleList([
            DecoderLayer(d_model, n_heads, d_ff, dropout_rate)
            for _ in range(n_layers)
        ])
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)


    def forward(self, x, memory):
        #print("Input tensor x:", x)
        #print("tok_embedding weight size:", self.tok_embedding.weight.size(0))

        # apply positional encoding to token sequences
        x = self.tok_embedding(x)  # (batch_size, target_seq_len, d_model)
        x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

        for layer in self.layers:
            x, attn = layer(x, memory)  # (batch_size, target_seq_len, d_model)

        x = self.layer_norm(x)  # (batch_size, target_seq_len, d_model)

        # x: (batch_size, target_seq_len, d_model)
        # attn: (batch_size, n_heads, target_seq_len, source_seq_len)
        return x, attn

In [ ]:
class Transformer(nn.Module):
    def __init__(self, encoder, decoder, num_classes, pad_idx, d_model):
        super(Transformer, self).__init__()
        self.pad_idx = pad_idx
        self.encoder = encoder
        self.decoder = decoder
        self.num_classes = num_classes
        self.fc_out = nn.Linear(d_model, self.num_classes)


    def forward(self, src, tgt):
        enc_output = self.encoder(src)
        dec_output, _ = self.decoder(tgt, enc_output)

        # Classification
        classification_output = dec_output[:, -1, :]  # Take the final output representation
        classification_output = self.fc_out(classification_output)  # Apply a linear layer
        #classification_output = nn.functional.softmax(classification_output, dim=-1)  # Apply softmax

        # classification_output: (batch_size, num_classes)
        return classification_output

Training

In [ ]:
input, target_index, vocab = Preprocessing(train_data_formatted)
val_input, val_target_index, val_vocab = Preprocessing(val_data_formatted)
test_input, test_target_index, test_vocab = Preprocessing(test_data_formatted)

In [ ]:
PAD_IDX = 0
INPUT_SIZE = len(vocab)
encoder = Encoder(INPUT_SIZE, HIDDEN_SIZE, N_LAYERS, N_HEADS, FF_SIZE, PAD_IDX, DROPOUT_RATE)
decoder = Decoder(INPUT_SIZE, HIDDEN_SIZE, N_LAYERS, N_HEADS, FF_SIZE, PAD_IDX, DROPOUT_RATE)

model = Transformer(encoder, decoder, OUTPUT_SIZE, PAD_IDX, HIDDEN_SIZE).to(device)

In [ ]:
def get_lr(step_num, warmup_steps=3000, d_model=512):
    step_num = max(1, step_num)
    arg1 = step_num ** -0.5
    arg2 = step_num * warmup_steps ** -1.5
    lrate = d_model ** -0.5 * min(arg1, arg2)
    return lrate


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=get_lr(0))


def calculate_accuracy(output, targets):
    predictions = output.argmax(dim=1, keepdim=True)  # assuming your output includes class scores
    correct = predictions.eq(targets.view_as(predictions)).sum().item()
    return correct / len(targets)


def train(early_stopping=True, patience=25):
    model.train() # Turn on the train mode
    step = 0
    losses = []
    accuracies = []
    val_losses = []
    val_accuracies = []
    best_val_loss = float('inf')
    best_model_state = None
    epochs_without_improvement = 0
    for epoch in range(1, N_EPOCHS + 1):
      total_loss = 0.0
      total_accuracy = 0.0
      for batch, i in enumerate(range(0, input.size(0) - 1, bptt)):
        data, targets = get_batch(input, torch.tensor(target_index), i, bptt)
        #print(data.size(), targets.size())
        #print(torch.isnan(data).any(), torch.isinf(data).any())
        #print(torch.isnan(targets).any(), torch.isinf(targets).any())

        optimizer.zero_grad()
        output = model(data, data)
        for name, param in model.named_parameters():
          if not torch.isfinite(param).all():
            print(f"Parameter {name} contains nan or inf.")
        if not torch.isfinite(output).all():
          print("Output contains nan or inf.")
        if not torch.isfinite(targets).all():
          print("Targets contain nan or inf.")
        #print("OPTSIZE: ", output.size())
        #print(output.view(-1, output.size(-1)).size(), targets.view(-1).size())
        loss = criterion(output.view(-1, output.size(-1)), targets.view(-1))
        #print(loss)
        loss.backward()
        w = torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)
        #print(w)
        for param_group in optimizer.param_groups:
            param_group['lr'] = get_lr(step + 1)

        optimizer.step()

        #print("STEP: ", step)
        step += 1

        total_loss += loss.item()
        total_accuracy += calculate_accuracy(output, targets)

      avg_loss = total_loss / len(range(0, input.size(0) - 1, bptt))
      avg_accuracy = total_accuracy / len(range(0, input.size(0) - 1, bptt))
      losses.append(avg_loss)
      accuracies.append(avg_accuracy)

      val_loss, val_accuracy = evaluate(model, val_input, val_target_index, criterion, bptt)
      val_losses.append(val_loss)
      val_accuracies.append(val_accuracy)

      print(f"Epoch {epoch}: Train Loss = {avg_loss}, Train Accuracy = {avg_accuracy}, Val Loss = {val_loss}, Val Accuracy = {val_accuracy}")

      # Early stopping
      if early_stopping:
          if val_loss < best_val_loss:
              best_val_loss = val_loss
              best_model_state = copy.deepcopy(model.state_dict())
              epochs_without_improvement = 0
          else:
              epochs_without_improvement += 1
              if epochs_without_improvement >= patience:
                  print(f"Early stopping after {epoch} epochs")
                  break

    if early_stopping:
        model.load_state_dict(best_model_state)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(losses, label='Training Loss')
    plt.title('Training Loss over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(accuracies, label='Training Accuracy')
    plt.title('Training Accuracy over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

    return model, losses, accuracies, val_losses, val_accuracies

# 3.Testing and Evaluation (Model A)
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [ ]:
def evaluate(model, test_input, test_target_index, criterion, bptt):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    total_accuracy = 0.0

    with torch.no_grad():  # Disable gradient tracking
        for i in range(0, test_input.size(0) - 1, bptt):
            data, targets = get_batch(test_input, torch.tensor(test_target_index), i, bptt)
            output = model(data, data)
            loss = criterion(output.view(-1, output.size(-1)), targets.view(-1))
            total_loss += loss.item()
            total_accuracy += calculate_accuracy(output, targets)

    avg_loss = total_loss / len(range(0, test_input.size(0) - 1, bptt))
    avg_accuracy = total_accuracy / len(range(0, test_input.size(0) - 1, bptt))

    return avg_loss, avg_accuracy

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def perf_eval(model, test_input, test_target_index, criterion, bptt):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    total_accuracy = 0.0

    all_targets = []
    all_predictions = []

    with torch.no_grad():  # Disable gradient tracking
        for i in range(0, test_input.size(0) - 1, bptt):
            data, targets = get_batch(test_input, torch.tensor(test_target_index), i, bptt)
            output = model(data, data)
            print(output.size())
            loss = criterion(output.view(-1, output.size(-1)), targets.view(-1))
            total_loss += loss.item()
            total_accuracy += calculate_accuracy(output, targets)

            # Collect targets and predictions for metrics calculation
            all_targets.extend(targets.view(-1).cpu().numpy())
            _, predicted = torch.max(output, 1)
            all_predictions.extend(predicted.view(-1).cpu().numpy())

    avg_loss = total_loss / len(range(0, test_input.size(0) - 1, bptt))
    avg_accuracy = total_accuracy / len(range(0, test_input.size(0) - 1, bptt))

    # Calculate precision, recall, F1 score, and MCC with zero_division set to 0
    precision = precision_score(all_targets, all_predictions, average='weighted', zero_division=0)
    recall = recall_score(all_targets, all_predictions, average='weighted', zero_division=0)
    f1 = f1_score(all_targets, all_predictions, average='weighted', zero_division=0)

    return avg_loss, avg_accuracy, precision, recall, f1

In [ ]:
model, train_losses, train_accuracies, val_losses, val_accuracies = train()

test_loss, test_accuracy, precision, recall, f1 = perf_eval(model, test_input, test_target_index, criterion, bptt)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")

Epoch 1: Train Loss = 1.1680684076266343, Train Accuracy = 0.3669943820224719, Val Loss = 1.0985509554545085, Val Accuracy = 0.4041666666666666
Epoch 2: Train Loss = 1.0573077697432443, Train Accuracy = 0.4502808988764047, Val Loss = 1.1112495164076488, Val Accuracy = 0.403125


KeyboardInterrupt: 

# 1. Dataset Processing (Model B)

The section immideately below contains all necessray imports.

In [ ]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import json
import copy

import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Get the file ID of the JSON file you want to open
test_id = "1PIaL_FR_NKabc-qKIi_RpaRwsoGgaPqw"
train_id = "1o3dgEDYno-d2WYM9w3tFE_8jF8pDYzyU"
val_id = "1TZZO6Pg16dI_flowwryrF0ZKfyBhrHDf"

# Download the file
test = drive.CreateFile({'id': test_id})
test.GetContentFile('test_data.json')
train = drive.CreateFile({'id': train_id})
train.GetContentFile('train_data.json')
val = drive.CreateFile({'id': val_id})
val.GetContentFile('val_data.json')

# Read the downloaded JSON file
with open('test_data.json', 'r') as f:
    test_data = json.load(f)

with open('train_data.json', 'r') as f:
    train_data = json.load(f)

with open('val_data.json', 'r') as f:
    val_data = json.load(f)


def format_rows(data):
    formatted_rows = []
    index = 0
    for row in data:
        sentence = row[0]
        aspect = row[1]
        polarity = row[2]
        formatted_rows.append([index, sentence, aspect, polarity])
        index += 1
    return formatted_rows


rows = test_data['data']
test_data_formatted = format_rows(rows)
rows = train_data['data']
train_data_formatted = format_rows(rows)
rows = val_data['data']
val_data_formatted = format_rows(rows)

print(val_data_formatted[0])

[0, 'I would wait for a table next time, the food was that good.', 'miscellaneous', 'neutral']


In [ ]:
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have",
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not",
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did",
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have",
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have",
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us",
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have",
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have",
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not",
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have",
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have",
                    "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will",
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have",
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have",
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}


def generate_ngrams(ipt, n):
    # Initialize the list to hold the n-grams
    opt = []
    for words in ipt:
        ngram_list = []
        # Generate n-grams with a step of 2 tokens before starting the next n-gram
        for i in range(0, len(words), 3):  # Step size of 3
            # Create the n-gram from position i to i+n
            if i + n <= len(words):
                ngram = words[i:i + n]
            else:
                # If we exceed the list length, pad the remaining elements with '<pad>'
                ngram = words[i:] + ['<pad>'] * (n - len(words[i:]))

            # Append the n-gram to the list
            ngram_list.append(ngram)

            # Break the loop when we have added the last possible n-gram that includes padding if needed
            if i + n >= len(words):
                break
        opt.append(ngram_list)
    return opt


def pre_process(sent_list, aspect):
    output = []
    for sent in sent_list:
        sent = sent.lower() #case-folding
        for word, new_word in contraction_dict.items():
            sent = sent.replace(word, new_word) #dealing with contractions
        sent = re.sub(r'[^\w\s]','',sent) #removing punctuation
        output.append(word_tokenize(sent)) #tokenization
    print(output)

    return generate_ngrams(output, 6)


def pre_process_decoder(sent_list):
    output = []
    for sent in sent_list:
        sent = sent.lower() #case-folding
        for word, new_word in contraction_dict.items():
            sent = sent.replace(word, new_word) #dealing with contractions
        sent = re.sub(r'[^\w\s]','',sent) #removing punctuation
        output.append(word_tokenize(sent)) #tokenization

    return output

In [ ]:
def to_index_T(data, to_ix):
    ground_truth = []
    for sent in data:
        ground_truth.append([to_ix[w] for w in sent])
    return ground_truth


def to_index_I(data, to_ix):
    input_index_list = []
    for sent in data:
      buffer = []
      for seq in sent:
        buffer.append([to_ix[w] for w in seq])
      input_index_list.append(buffer)
    return input_index_list


def make_batch(data):
    #cap_ext = max(len(lst) for lst in data)
    cap_ext = batch_size
    cap_int = []
    for x in data:
      cap_int.append(max(len(lst) for lst in x))
    cap_int = max(cap_int)
    print(cap_int, cap_ext)

    opt = []
    for module in data:
      if len(module) < cap_ext:
        w = []
        for _ in range(cap_ext-len(module)):
          w.append([0, 0])
        opt.append(module + w)
      else:
        opt.append(module)

    #print(opt[0])

    padded = []
    for module in opt:
      buffer = []
      for x in module:
        if len(x) < cap_int:
          w = []
          for _ in range(cap_int-len(x)):
            w.append(0)
          buffer.append(x + w)
        else:
          buffer.append(x)
      padded.append(buffer)

    #print(padded[0])
    padded = torch.tensor(padded)

    return padded.to(device)


def make_batch_decoder(data):
    cap = max(len(lst) for lst in data)
    #cap = batch_size
    padded = []
    for x in data:
      if len(x) < cap:
        w = []
        for _ in range(cap-len(x)):
          w.append(0)
        padded.append(x + w)
      else:
        padded.append(x)

    padded = torch.tensor(padded)

    return padded.to(device)


def get_batch(ipt, lbl, i, bptt):
    seq_len = min(bptt, len(ipt) - 1 - i)
    data = ipt[i:i+seq_len].to(device)
    target = lbl[i:i+seq_len].to(device)

    if len(data) < bptt:
        pad_len = bptt - len(data)
        pad_data = torch.zeros(pad_len, data.size(1), data.size(2), dtype=data.dtype).fill_(0).to(device)
        data = torch.cat([data, pad_data], dim=0)
        pad_target = torch.zeros(pad_len, target.size(1), dtype=target.dtype).fill_(0).to(device)
        target = torch.cat([target, pad_target], dim=0)

    return data, target


def get_batch_decoder(ipt, lbl, i, bptt):
    seq_len = min(bptt, len(ipt) - 1 - i)
    data = ipt[i:i+seq_len].to(device)
    target = lbl[i:i+seq_len].to(device)

    if len(data) < bptt:
        pad_len = bptt - len(data)
        pad_data = torch.zeros(pad_len, data.size(1), dtype=data.dtype).fill_(0).to(device)
        data = torch.cat([data, pad_data], dim=0)
        pad_target = torch.zeros(pad_len, target.size(1), dtype=target.dtype).fill_(0).to(device)
        target = torch.cat([target, pad_target], dim=0)

    return data, target

In [ ]:
def Preprocessing(Source):
    sentences = []
    aspects = []
    labels = []
    for row in Source:
      sentences.append(row[1])
      aspects.append(row[2])
      labels.append(row[3])

    # Preprocessing the data using the function defined above
    input_token_list_decoder = pre_process_decoder(sentences) # -> input to encoder

    input_token_list_n = []   ##MOD
    for x in range(len(input_token_list_decoder)):    ##MOD
      input_token_list_n.append(input_token_list_decoder[x] + ["<HL>"] + [aspects[x]])    ##MOD

    # Preprocessing the data using the function defined above
    input_token_list = pre_process(sentences, aspects) # -> input to encoder
    label_token_list = pre_process_decoder(labels)
    print(input_token_list[0], label_token_list[0])
    print(len(input_token_list), len(label_token_list))

    target_token_list = label_token_list

    # set up a vocab to index dictionary
    word_to_ix = {"<pad>": 0, "<HL>": 1}    ##MOD
    for module in input_token_list:
      for sentence in module:
        for word in sentence:
            if word not in word_to_ix:
                word_to_ix[word] = len(word_to_ix)

    for sentence in input_token_list_n:
        for word in sentence:
            if word not in word_to_ix:
              word_to_ix[word] = len(word_to_ix)

    word_list = list(word_to_ix.keys())
    print(word_to_ix)
    print(len(word_to_ix))

    label_to_index = {
        "positive": 0,
        "negative": 1,
        "neutral": 2
    }

    input_index = to_index_I(input_token_list, word_to_ix)
    print(input_index[0])
    input_index_decoder = to_index_T(input_token_list_n, word_to_ix)    ##MOD
    target_index = to_index_T(target_token_list, label_to_index)

    input = make_batch(input_index)
    input_decoder = make_batch_decoder(input_index_decoder)
    print(input.size(), input_decoder.size())

    return input, input_decoder, target_index, len(word_to_ix)

# 2. Model Implementation (Model B)

In [ ]:
OUTPUT_SIZE = 3  # target vocab size
HIDDEN_SIZE = 512
N_LAYERS = 1
N_encoder_LAYERS = 1
N_ca_LAYERS = 1
N_HEADS = 8
FF_SIZE = 2048
DROPOUT_RATE = 0.1
CLIP = 1
N_EPOCHS = 50
batch_size = 80
bptt = batch_size
print(batch_size)

In [ ]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, scale, dropout_rate):
        super(ScaledDotProductAttention, self).__init__()
        self.scale = scale
        self.dropout_rate = dropout_rate

        self.dropout = nn.Dropout(dropout_rate)


    def forward(self, query, key, value):
        # calculate alignment scores
        scores = torch.matmul(query, key.transpose(-2, -1))  # (batch_size, n_heads, query_len, value_len)
        scores = scores / self.scale  # (batch_size, num_heads, query_len, value_len)

        # calculate the attention weights (prob) from alignment scores
        attn_probs = F.softmax(scores, dim=-1)  # (batch_size, n_heads, query_len, value_len)

        # calculate context vector
        output = torch.matmul(self.dropout(attn_probs), value)  # (batch_size, n_heads, query_len, head_dim)

        # output: (batch_size, n_heads, query_len, head_dim)
        # attn_probs: (batch_size, n_heads, query_len, value_len)
        return output, attn_probs

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout_rate=0.1):
        super(MultiHeadAttention, self).__init__()
        assert d_model % n_heads == 0, "`d_model` should be a multiple of `n_heads`"

        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = self.d_v = d_model // n_heads  # head_dim
        self.dropout_rate = dropout_rate

        self.W_q = nn.Linear(d_model, d_model, bias=False)  # linear transformation for the Q, K and V vectors
        self.W_k = nn.Linear(d_model, d_model, bias=False)
        self.W_v = nn.Linear(d_model, d_model, bias=False)
        self.W_o = nn.Linear(d_model, d_model)
        self.attention = ScaledDotProductAttention(np.sqrt(self.d_k), dropout_rate)


    def project(self, x):
        batch_size = x.size(0)
        x = x.view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)  # (batch_size, n_heads, seq_len, d_k)

        return x


    def unproject(self, x):
        batch_size = x.size(0)
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.n_heads * self.d_k)

        return x


    def forward(self, query, key, value):
        # apply linear projections to query, key and value
        Q = self.project(self.W_q(query))  # (batch_size, n_heads, query_len, head_dim)
        K = self.project(self.W_k(key))  # (batch_size, n_heads, key_len, head_dim)
        V = self.project(self.W_v(value))  # (batch_size, n_heads, value_len, head_dim)

        # calculate attention weights and context vector for each of the heads
        x, attn = self.attention(Q, K, V)

        # concatenate context vector of all the heads
        x = self.unproject(x)  # (batch_size, query_len, d_model)

        # apply linear projection to concatenated context vector
        x = self.W_o(x)

        return x, attn

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, vocab_size=6000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        encoding = torch.zeros(vocab_size, d_model)
        position = torch.arange(0, vocab_size, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float()
            * (-math.log(10000.0) / d_model)
        )
        encoding[:, 0::2] = torch.sin(position * div_term)
        encoding[:, 1::2] = torch.cos(position * div_term)
        encoding = encoding.unsqueeze(0)
        self.register_buffer("encoding", encoding)


    def forward(self, x):
        x = x + self.encoding[:, : x.size(1), :]
        x = self.dropout(x)
        return x

In [ ]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout_rate=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff
        self.dropout_rate = dropout_rate

        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout_rate)


    def forward(self, x):
        x = self.dropout(F.relu(self.w_1(x)))  # (batch_size, seq_len, d_ff)
        x = self.w_2(x)  # (batch_size, seq_len, d_model)

        # x: (batch_size, seq_len, d_model)
        return x

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout_rate=0.1):
        super(EncoderLayer, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff
        self.n_heads = n_heads
        self.dropout_rate = dropout_rate

        self.attn_layer = MultiHeadAttention(d_model, n_heads, dropout_rate)
        self.attn_layer_norm = nn.LayerNorm(d_model, eps=1e-6)

        self.ff_layer = PositionwiseFeedForward(d_model, d_ff, dropout_rate)
        self.ff_layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout = nn.Dropout(dropout_rate)


    def forward(self, x):
        # apply self-attention
        x1, _ = self.attn_layer(x, x, x)  # (batch_size, source_seq_len, d_model)

        # apply residual connection followed by layer normalization
        x = self.attn_layer_norm(x + self.dropout(x1))  # (batch_size, source_seq_len, d_model)

        #x1 = self.ff_layer(x)  # (batch_size, source_seq_len, d_model)
        #x = self.ff_layer_norm(x + self.dropout(x1))  # (batch_size, source_seq_len, d_model)

        return x

In [ ]:
class CrossAttentionEncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout_rate=0.1):
        super(CrossAttentionEncoderLayer, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff
        self.n_heads = n_heads
        self.dropout_rate = dropout_rate

        self.attn_layer = MultiHeadAttention(d_model, n_heads, dropout_rate)
        self.attn_layer_norm = nn.LayerNorm(d_model, eps=1e-6)

        self.ff_layer = PositionwiseFeedForward(d_model, d_ff, dropout_rate)
        self.ff_layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout = nn.Dropout(dropout_rate)


    def forward(self, x):
        # apply self-attention
        x1, _ = self.attn_layer(x, x, x)  # (batch_size, source_seq_len, d_model)

        # apply residual connection followed by layer normalization
        x = self.attn_layer_norm(x + self.dropout(x1))  # (batch_size, source_seq_len, d_model)

        # apply position-wise feed-forward
        x1 = self.ff_layer(x)  # (batch_size, source_seq_len, d_model)

        # apply residual connection followed by layer normalization
        x = self.ff_layer_norm(x + self.dropout(x1))  # (batch_size, source_seq_len, d_model)

        return x

In [ ]:
class CrossAttentionEncoder(nn.Module):
    def __init__(self, vocab_size, d_model, n_encoder_layers, n_heads, d_ff, pad_idx, dropout_rate=0.1, max_len=5000):
        super(CrossAttentionEncoder, self).__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.n_encoder_layers = n_encoder_layers
        self.d_ff = d_ff
        self.pad_idx = pad_idx
        self.dropout_rate = dropout_rate
        self.max_len = max_len
        self.layers = nn.ModuleList([
            CrossAttentionEncoderLayer(d_model, n_heads, d_ff, dropout_rate)
            for _ in range(n_encoder_layers)
        ])
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)


    def forward(self, x):
        x = self.layer_norm(x)

        for layer in self.layers:
            x = layer(x)  # (batch_size, source_seq_len, d_model)

        x = self.layer_norm(x)  # (batch_size, source_seq_len, d_model)

        return x

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, n_encoder_layers, n_heads, d_ff, pad_idx, dropout_rate=0.1, max_len=5000):
        super(Encoder, self).__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.n_encoder_layers = n_encoder_layers
        self.d_ff = d_ff
        self.pad_idx = pad_idx
        self.dropout_rate = dropout_rate
        self.max_len = max_len

        self.tok_embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_embedding = PositionalEncoding(d_model, dropout_rate, max_len)
        self.layers = nn.ModuleList([
            EncoderLayer(d_model, n_heads, d_ff, dropout_rate)
            for _ in range(n_encoder_layers)
        ])
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)


    def forward(self, x):
        # apply positional encoding to token sequences
        x = self.tok_embedding(x)  # (batch_size, source_seq_len, d_model)
        x = self.pos_embedding(x)  # (batch_size, source_seq_len, d_model)

        for layer in self.layers:
            x = layer(x)  # (batch_size, source_seq_len, d_model)

        x = self.layer_norm(x)  # (batch_size, source_seq_len, d_model)

        return x

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout_rate=0.1, num_classes=None):
        super(DecoderLayer, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff
        self.n_heads = n_heads
        self.dropout_rate = dropout_rate
        self.attn_layer = MultiHeadAttention(d_model, n_heads, dropout_rate)
        self.attn_layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.enc_attn_layer = MultiHeadAttention(d_model, n_heads, dropout_rate)
        self.enc_attn_layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.ff_layer = PositionwiseFeedForward(d_model, d_ff, dropout_rate)
        self.ff_layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout = nn.Dropout(dropout_rate)

        # Classification layer
        self.num_classes = num_classes
        if self.num_classes is not None:
            self.classifier = nn.Linear(d_model, num_classes)

    def forward(self, x, memory):
        # apply self-attention
        x1, _ = self.attn_layer(x, x, x)
        # (batch_size, target_seq_len, d_model)
        # apply residual connection followed by layer normalization
        x = self.attn_layer_norm(x + self.dropout(x1))
        # (batch_size, target_seq_len, d_model)

        # memory is the output from encoder block (encoder states)
        x1, attn = self.enc_attn_layer(x, memory, memory)
        # x1: (batch_size, target_seq_len, d_model)
        # attn: (batch_size, n_heads, target_seq_len, source_seq_len)
        # apply residual connection followed by layer normalization
        x = self.enc_attn_layer_norm(x + self.dropout(x1))
        # (batch_size, target_seq_len, d_model)

        # apply position-wise feed-forward
        x1 = self.ff_layer(x)
        # (batch_size, target_seq_len, d_model)
        # apply residual connection followed by layer normalization
        x = self.ff_layer_norm(x + self.dropout(x1))
        # (batch_size, target_seq_len, d_model)

        return x, attn

In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, n_layers, n_heads, d_ff, pad_idx, dropout_rate=0.1, max_len=5000):
        super(Decoder, self).__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.n_layers = n_layers
        self.d_ff = d_ff
        self.pad_idx = pad_idx
        self.dropout_rate = dropout_rate
        self.max_len = max_len

        self.tok_embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_embedding = PositionalEncoding(d_model, dropout_rate, max_len)
        self.layers = nn.ModuleList([
            DecoderLayer(d_model, n_heads, d_ff, dropout_rate)
            for _ in range(n_layers)
        ])
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)


    def forward(self, x, memory):
        #print("Input tensor x:", x)
        #print("tok_embedding weight size:", self.tok_embedding.weight.size(0))

        # apply positional encoding to token sequences
        x = self.tok_embedding(x)  # (batch_size, target_seq_len, d_model)
        x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

        for layer in self.layers:
            x, attn = layer(x, memory)  # (batch_size, target_seq_len, d_model)

        x = self.layer_norm(x)  # (batch_size, target_seq_len, d_model)

        return x, attn

In [ ]:
class Transformer(nn.Module):
    def __init__(self, encoder, decoder, ca_encoder, num_classes, pad_idx, d_model):
        super(Transformer, self).__init__()
        self.pad_idx = pad_idx
        self.encoder = encoder
        self.decoder = decoder
        self.ca_encoder = ca_encoder
        self.num_classes = num_classes
        self.fc_out = nn.Linear(d_model, self.num_classes)


    def forward(self, src, tgt):
        enc_output = self.encoder(src[0])

        for module in src[1:]:
          enc_output += self.encoder(module)

        enc_output = self.ca_encoder(enc_output)
        #print("CAEN SHAPE: ", enc_output.size())
        dec_output, _ = self.decoder(tgt, enc_output)

        # Classification
        classification_output = dec_output[:, -1, :]  # Take the final output representation
        classification_output = self.fc_out(classification_output)  # Apply a linear layer
        #classification_output = nn.functional.softmax(classification_output, dim=-1)  # Apply softmax

        return classification_output

In [ ]:
input, input_decoder, target_index, vocab_size = Preprocessing(train_data_formatted)
val_input, val_input_decoder, val_target_index, val_vocab_size = Preprocessing(val_data_formatted)
test_input, test_input_decoder, test_target_index, test_vocab_size = Preprocessing(test_data_formatted)

In [ ]:
PAD_IDX = 0
INPUT_SIZE = vocab_size
encoder = Encoder(INPUT_SIZE, HIDDEN_SIZE, N_encoder_LAYERS, N_HEADS, FF_SIZE, PAD_IDX, DROPOUT_RATE)
decoder = Decoder(INPUT_SIZE, HIDDEN_SIZE, N_LAYERS, N_HEADS, FF_SIZE, PAD_IDX, DROPOUT_RATE)
ca_encoder = CrossAttentionEncoder(INPUT_SIZE, HIDDEN_SIZE, N_ca_LAYERS, N_HEADS, FF_SIZE, PAD_IDX, DROPOUT_RATE)

model = Transformer(encoder, decoder, ca_encoder, OUTPUT_SIZE, PAD_IDX, HIDDEN_SIZE).to(device)

In [ ]:
def get_lr(step_num, warmup_steps=3000, d_model=512):
    step_num = max(1, step_num)
    arg1 = step_num ** -0.5
    arg2 = step_num * warmup_steps ** -1.5
    lrate = d_model ** -0.5 * min(arg1, arg2)
    return lrate


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=get_lr(0))

losses = []
accuracies = []


def calculate_accuracy(output, targets):
    predictions = output.argmax(dim=1, keepdim=True)  # assuming your output includes class scores
    correct = predictions.eq(targets.view_as(predictions)).sum().item()
    return correct / len(targets)


def train(early_stopping=True, patience=25):
    model.train() # Turn on the train mode
    step = 0
    losses = []
    accuracies = []
    val_losses = []
    val_accuracies = []
    best_val_loss = float('inf')
    best_model_state = None
    epochs_without_improvement = 0
    for epoch in range(1, N_EPOCHS + 1):
      total_loss = 0.0
      total_accuracy = 0.0
      #print("INPUT SIZE: ", input.size(0))
      for batch, i in enumerate(range(0, input.size(0) - 1, bptt)):
        #print("I: ", i)
        data, targets = get_batch(input, torch.tensor(target_index), i, bptt)
        data_decoder, _ = get_batch_decoder(input_decoder, torch.tensor(target_index), i, bptt)
        #print(data.size(), targets.size(), data_decoder.size())
        #print(data, targets)
        #print(torch.isnan(data).any(), torch.isinf(data).any())
        #print(torch.isnan(targets).any(), torch.isinf(targets).any())

        optimizer.zero_grad()
        output = model(data, data_decoder)
        for name, param in model.named_parameters():
          if not torch.isfinite(param).all():
            print(f"Parameter {name} contains nan or inf.")
        if not torch.isfinite(output).all():
          print("Output contains nan or inf.")
        if not torch.isfinite(targets).all():
          print("Targets contain nan or inf.")
        #print("OPTSIZE: ", output.size())
        #print(output.view(-1, output.size(-1)).size(), targets.view(-1).size())
        loss = criterion(output.view(-1, output.size(-1)), targets.view(-1))
        #print(loss)
        loss.backward()
        w = torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)
        #print(w)

        for param_group in optimizer.param_groups:
            param_group['lr'] = get_lr(step + 1)

        optimizer.step()

        #print("STEP: ", step)
        step += 1

        total_loss += loss.item()
        total_accuracy += calculate_accuracy(output, targets)

      avg_loss = total_loss / len(range(0, input.size(0) - 1, bptt))
      avg_accuracy = total_accuracy / len(range(0, input.size(0) - 1, bptt))
      losses.append(avg_loss)
      accuracies.append(avg_accuracy)

      val_loss, val_accuracy = evaluate(model, val_input, val_input_decoder, val_target_index, criterion, bptt)
      val_losses.append(val_loss)
      val_accuracies.append(val_accuracy)

      print(f"Epoch {epoch}: Train Loss = {avg_loss}, Train Accuracy = {avg_accuracy}, Val Loss = {val_loss}, Val Accuracy = {val_accuracy}")

      # Early stopping
      if early_stopping:
          if val_loss < best_val_loss:
              best_val_loss = val_loss
              best_model_state = copy.deepcopy(model.state_dict())
              epochs_without_improvement = 0
          else:
              epochs_without_improvement += 1
              if epochs_without_improvement >= patience:
                  print(f"Early stopping after {epoch} epochs")
                  break

    if early_stopping:
        model.load_state_dict(best_model_state)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(losses, label='Training Loss')
    plt.title('Training Loss over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(accuracies, label='Training Accuracy')
    plt.title('Training Accuracy over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

    return model, losses, accuracies, val_losses, val_accuracies

# Testing and Evaluation (Model B)

In [ ]:
def evaluate(model, test_input, test_input_decoder, test_target_index, criterion, bptt):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    total_accuracy = 0.0

    with torch.no_grad():  # Disable gradient tracking
        for i in range(0, test_input.size(0) - 1, bptt):
            data, targets = get_batch(test_input, torch.tensor(test_target_index), i, bptt)
            data_decoder, _ = get_batch_decoder(test_input_decoder, torch.tensor(test_target_index), i, bptt)
            output = model(data, data_decoder)
            loss = criterion(output.view(-1, output.size(-1)), targets.view(-1))
            total_loss += loss.item()
            total_accuracy += calculate_accuracy(output, targets)

    avg_loss = total_loss / len(range(0, test_input.size(0) - 1, bptt))
    avg_accuracy = total_accuracy / len(range(0, test_input.size(0) - 1, bptt))

    return avg_loss, avg_accuracy

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def perf_eval(model, test_input, test_input_decoder, test_target_index, criterion, bptt):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    total_accuracy = 0.0
    all_predictions = []
    all_targets = []

    with torch.no_grad():  # Disable gradient tracking
        for i in range(0, test_input.size(0) - 1, bptt):
            data, targets = get_batch(test_input, torch.tensor(test_target_index), i, bptt)
            data_decoder, _ = get_batch_decoder(test_input_decoder, torch.tensor(test_target_index), i, bptt)
            output = model(data, data_decoder)
            loss = criterion(output.view(-1, output.size(-1)), targets.view(-1))
            total_loss += loss.item()
            total_accuracy += calculate_accuracy(output, targets)

            predictions = output.argmax(dim=-1).cpu().numpy().flatten()
            all_predictions.extend(predictions)
            all_targets.extend(targets.cpu().numpy().flatten())

    avg_loss = total_loss / len(range(0, test_input.size(0) - 1, bptt))
    avg_accuracy = total_accuracy / len(range(0, test_input.size(0) - 1, bptt))
    precision = precision_score(all_targets, all_predictions, average='weighted', zero_division=0)
    recall = recall_score(all_targets, all_predictions, average='weighted', zero_division=0)
    f1 = f1_score(all_targets, all_predictions, average='weighted', zero_division=0)

    return avg_loss, avg_accuracy, precision, recall, f1

In [ ]:
model, train_losses, train_accuracies, val_losses, val_accuracies = train()

test_loss, test_accuracy, precision, recall, f1 = perf_eval(model, test_input, test_input_decoder, test_target_index, criterion, bptt)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")

# Dataset Processing (Model C)

In [ ]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import json
import copy

import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Set the device for PyTorch (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Get the file ID of the JSON file you want to open
test_id = "1PIaL_FR_NKabc-qKIi_RpaRwsoGgaPqw"
train_id = "1o3dgEDYno-d2WYM9w3tFE_8jF8pDYzyU"
val_id = "1TZZO6Pg16dI_flowwryrF0ZKfyBhrHDf"

# Download the file
test = drive.CreateFile({'id': test_id})
test.GetContentFile('test_data.json')
train = drive.CreateFile({'id': train_id})
train.GetContentFile('train_data.json')
val = drive.CreateFile({'id': val_id})
val.GetContentFile('val_data.json')

# Read the downloaded JSON file
with open('test_data.json', 'r') as f:
    test_data = json.load(f)

with open('train_data.json', 'r') as f:
    train_data = json.load(f)

with open('val_data.json', 'r') as f:
    val_data = json.load(f)


def format_rows(data):
    formatted_rows = []
    index = 0
    for row in data:
        sentence = row[0]
        aspect = row[1]
        polarity = row[2]
        formatted_rows.append([index, sentence, aspect, polarity])
        index += 1
    return formatted_rows


rows = test_data['data']
test_data_formatted = format_rows(rows)
rows = train_data['data']
train_data_formatted = format_rows(rows)
rows = val_data['data']
val_data_formatted = format_rows(rows)

print(val_data_formatted[0])

def unique_pairs(data_list):
    # A set to hold unique elements
    unique_elements = set()

    # Iterate over each sublist in the main list
    for entry in data_list:
        # Check if the sublist is long enough to have a second element
        if len(entry) > 1:
            # Add the second element to the set
            unique_elements.add(entry[1])

    opt = {}
    for unit in unique_elements:
        matching_entries = set()

        for entry in data_list:
            # Check if the sublist is long enough and if the second element matches the target sentence
            if len(entry) > 1 and entry[1] == unit:
                matching_entries.add(entry[2])

        opt[unit] = matching_entries

    return opt

In [ ]:
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have",
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not",
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did",
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have",
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have",
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us",
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have",
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have",
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not",
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have",
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have",
                    "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will",
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have",
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have",
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}


def set_to_string(elements_set):
    # Convert each element in the set to a string (in case there are non-string types)
    elements_set = map(str, elements_set)

    # Join elements with a space as the separator
    result_string = ' '.join(elements_set)

    return result_string


def pre_process(sent_list, ap):
    output = []
    for unit in sent_list:
        sent = unit.lower() #case-folding
        for word, new_word in contraction_dict.items():
            sent = sent.replace(word, new_word) #dealing with contractions
        sent = re.sub(r'[^\w\s]','',sent) #removing punctuation
        front = word_tokenize(sent)
        back = word_tokenize(set_to_string(ap[unit]).lower())
        opt = front + ["<HL>"] + back
        output.append(opt)
    return output


def pre_process_l(sent_list):
    output = []
    for sent in sent_list:
        sent = sent.lower() #case-folding
        for word, new_word in contraction_dict.items():
            sent = sent.replace(word, new_word) #dealing with contractions
        sent = re.sub(r'[^\w\s]','',sent) #removing punctuation
        output.append(word_tokenize(sent)) #tokenization
    return output

In [ ]:
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list


def make_batch(data):
    #cap = max(len(lst) for lst in data)
    cap = 80    # This should be equal to batch_size
    padded = []
    for x in data:
      if len(x) < cap:
        w = []
        for _ in range(cap-len(x)):
          w.append(0)
        padded.append(x + w)
      else:
        padded.append(x)

    padded = torch.tensor(padded)

    return padded.to(device)


def get_batch(ipt, lbl, i, bptt):
    seq_len = min(bptt, len(ipt) - 1 - i)
    data = ipt[i:i+seq_len].to(device)
    target = lbl[i:i+seq_len].to(device)

    if len(data) < bptt:
        pad_len = bptt - len(data)
        pad_data = torch.zeros(pad_len, data.size(1), dtype=data.dtype).fill_(0).to(device)
        data = torch.cat([data, pad_data], dim=0)
        pad_target = torch.zeros(pad_len, target.size(1), dtype=target.dtype).fill_(0).to(device)
        target = torch.cat([target, pad_target], dim=0)

    return data, target

In [ ]:
def Preprocessing(Source):
    sentences = []
    aspects = []
    labels = []
    for row in Source:
      sentences.append(row[1])
      aspects.append(row[2])
      labels.append(row[3])

    ap = unique_pairs(Source)

    # Preprocessing the data using the function defined above
    input_token_list = pre_process(sentences, ap) # -> input to encoder
    label_token_list = pre_process_l(labels)

    #print(input_token_list)

    output_token_list = label_token_list
    target_token_list = label_token_list

    # set up a vocab to index dictionary
    word_to_ix = {"<pad>": 0, "<HL>": 1}
    for sentence in input_token_list+output_token_list:
        for word in sentence:
            if word not in word_to_ix:
                word_to_ix[word] = len(word_to_ix)
    word_list = list(word_to_ix.keys())
    #print(word_to_ix)
    #print(len(word_to_ix))

    label_to_index = {
        "positive": 0,
        "negative": 1,
        "neutral": 2
    }

    input_index = to_index(input_token_list, word_to_ix)
    target_index = to_index(target_token_list, label_to_index)

    input = make_batch(input_index)
    #print(input.size())

    return input, target_index, len(word_to_ix)

In [ ]:
input, target_index, vocab_size = Preprocessing(train_data_formatted)
val_input, val_target_index, val_vocab_size = Preprocessing(val_data_formatted)
test_input, test_target_index, test_vocab_size = Preprocessing(test_data_formatted)

# Model Implementation (Model C)

In [ ]:
INPUT_SIZE = vocab_size  # source vocab size
OUTPUT_SIZE = 3  # target vocab size
HIDDEN_SIZE = 512
N_LAYERS = 1
N_HEADS = 8
FF_SIZE = 2048
DROPOUT_RATE = 0.6
CLIP = 1
N_EPOCHS = 50
batch_size = 80
bptt = batch_size
print(batch_size)

In [ ]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, scale, dropout_rate):
        super(ScaledDotProductAttention, self).__init__()
        self.scale = scale
        self.dropout_rate = dropout_rate

        self.dropout = nn.Dropout(dropout_rate)


    def forward(self, query, key, value):
        # calculate alignment scores
        scores = torch.matmul(query, key.transpose(-2, -1))  # (batch_size, n_heads, query_len, value_len)
        scores = scores / self.scale  # (batch_size, num_heads, query_len, value_len)

        # calculate the attention weights (prob) from alignment scores
        attn_probs = F.softmax(scores, dim=-1)  # (batch_size, n_heads, query_len, value_len)

        # calculate context vector
        output = torch.matmul(self.dropout(attn_probs), value)  # (batch_size, n_heads, query_len, head_dim)

        # output: (batch_size, n_heads, query_len, head_dim)
        # attn_probs: (batch_size, n_heads, query_len, value_len)
        return output, attn_probs

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout_rate=0.1):
        super(MultiHeadAttention, self).__init__()
        assert d_model % n_heads == 0, "`d_model` should be a multiple of `n_heads`"

        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = self.d_v = d_model // n_heads  # head_dim
        self.dropout_rate = dropout_rate

        self.W_q = nn.Linear(d_model, d_model, bias=False)  # linear transformation for the Q, K and V vectors
        self.W_k = nn.Linear(d_model, d_model, bias=False)
        self.W_v = nn.Linear(d_model, d_model, bias=False)
        self.W_o = nn.Linear(d_model, d_model)
        self.attention = ScaledDotProductAttention(np.sqrt(self.d_k), dropout_rate)


    def project(self, x):
        batch_size = x.size(0)
        x = x.view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)  # (batch_size, n_heads, seq_len, d_k)

        return x


    def unproject(self, x):
        batch_size = x.size(0)
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.n_heads * self.d_k)

        return x


    def forward(self, query, key, value):
        # apply linear projections to query, key and value
        Q = self.project(self.W_q(query))  # (batch_size, n_heads, query_len, head_dim)
        K = self.project(self.W_k(key))  # (batch_size, n_heads, key_len, head_dim)
        V = self.project(self.W_v(value))  # (batch_size, n_heads, value_len, head_dim)

        # calculate attention weights and context vector for each of the heads
        x, attn = self.attention(Q, K, V)

        # concatenate context vector of all the heads
        x = self.unproject(x)  # (batch_size, query_len, d_model)

        # apply linear projection to concatenated context vector
        x = self.W_o(x)

        return x, attn

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, vocab_size=6000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        encoding = torch.zeros(vocab_size, d_model)
        position = torch.arange(0, vocab_size, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float()
            * (-math.log(10000.0) / d_model)
        )
        encoding[:, 0::2] = torch.sin(position * div_term)
        encoding[:, 1::2] = torch.cos(position * div_term)
        encoding = encoding.unsqueeze(0)
        self.register_buffer("encoding", encoding)


    def forward(self, x):
        x = x + self.encoding[:, : x.size(1), :]
        x = self.dropout(x)
        return x

In [ ]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout_rate=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff
        self.dropout_rate = dropout_rate

        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout_rate)


    def forward(self, x):
        x = self.dropout(F.relu(self.w_1(x)))  # (batch_size, seq_len, d_ff)
        x = self.w_2(x)  # (batch_size, seq_len, d_model)

        # x: (batch_size, seq_len, d_model)
        return x

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout_rate=0.1):
        super(EncoderLayer, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff
        self.n_heads = n_heads
        self.dropout_rate = dropout_rate

        self.attn_layer = MultiHeadAttention(d_model, n_heads, dropout_rate)
        self.attn_layer_norm = nn.LayerNorm(d_model, eps=1e-6)

        self.ff_layer = PositionwiseFeedForward(d_model, d_ff, dropout_rate)
        self.ff_layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout = nn.Dropout(dropout_rate)


    def forward(self, x):
        # apply self-attention
        x1, _ = self.attn_layer(x, x, x)  # (batch_size, source_seq_len, d_model)

        # apply residual connection followed by layer normalization
        x = self.attn_layer_norm(x + self.dropout(x1))  # (batch_size, source_seq_len, d_model)

        # apply position-wise feed-forward
        x1 = self.ff_layer(x)  # (batch_size, source_seq_len, d_model)

        # apply residual connection followed by layer normalization
        x = self.ff_layer_norm(x + self.dropout(x1))  # (batch_size, source_seq_len, d_model)

        return x

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, n_layers, n_heads, d_ff, pad_idx, dropout_rate=0.1, max_len=5000):
        super(Encoder, self).__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.n_layers = n_layers
        self.d_ff = d_ff
        self.pad_idx = pad_idx
        self.dropout_rate = dropout_rate
        self.max_len = max_len

        self.tok_embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_embedding = PositionalEncoding(d_model, dropout_rate, max_len)
        self.layers = nn.ModuleList([
            EncoderLayer(d_model, n_heads, d_ff, dropout_rate)
            for _ in range(n_layers)
        ])
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)


    def forward(self, x):
        # apply positional encoding to token sequences
        x = self.tok_embedding(x)  # (batch_size, source_seq_len, d_model)
        x = self.pos_embedding(x)  # (batch_size, source_seq_len, d_model)

        for layer in self.layers:
            x = layer(x)  # (batch_size, source_seq_len, d_model)

        x = self.layer_norm(x)  # (batch_size, source_seq_len, d_model)

        return x

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout_rate=0.1, num_classes=None):
        super(DecoderLayer, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff
        self.n_heads = n_heads
        self.dropout_rate = dropout_rate
        self.attn_layer = MultiHeadAttention(d_model, n_heads, dropout_rate)
        self.attn_layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.enc_attn_layer = MultiHeadAttention(d_model, n_heads, dropout_rate)
        self.enc_attn_layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.ff_layer = PositionwiseFeedForward(d_model, d_ff, dropout_rate)
        self.ff_layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout = nn.Dropout(dropout_rate)

        # Classification layer
        self.num_classes = num_classes
        if self.num_classes is not None:
            self.classifier = nn.Linear(d_model, num_classes)

    def forward(self, x, memory):
        # apply self-attention
        x1, _ = self.attn_layer(x, x, x)

        x = self.attn_layer_norm(x + self.dropout(x1))

        x1, attn = self.enc_attn_layer(x, memory, memory)

        x = self.enc_attn_layer_norm(x + self.dropout(x1))

        x1 = self.ff_layer(x)

        x = self.ff_layer_norm(x + self.dropout(x1))

        return x, attn

In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, n_layers, n_heads, d_ff, pad_idx, dropout_rate=0.1, max_len=5000):
        super(Decoder, self).__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.n_layers = n_layers
        self.d_ff = d_ff
        self.pad_idx = pad_idx
        self.dropout_rate = dropout_rate
        self.max_len = max_len

        self.tok_embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_embedding = PositionalEncoding(d_model, dropout_rate, max_len)
        self.layers = nn.ModuleList([
            DecoderLayer(d_model, n_heads, d_ff, dropout_rate)
            for _ in range(n_layers)
        ])
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)


    def forward(self, x, memory):
        #print("Input tensor x:", x)
        #print("tok_embedding weight size:", self.tok_embedding.weight.size(0))

        # apply positional encoding to token sequences
        x = self.tok_embedding(x)  # (batch_size, target_seq_len, d_model)
        x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

        for layer in self.layers:
            x, attn = layer(x, memory)  # (batch_size, target_seq_len, d_model)

        x = self.layer_norm(x)  # (batch_size, target_seq_len, d_model)

        return x, attn

In [ ]:
class Transformer(nn.Module):
    def __init__(self, encoder, decoder, num_classes, pad_idx, d_model):
        super(Transformer, self).__init__()
        self.pad_idx = pad_idx
        self.encoder = encoder
        self.decoder = decoder
        self.num_classes = num_classes
        self.fc_out = nn.Linear(d_model, self.num_classes)


    def forward(self, src, tgt):
        enc_output = self.encoder(src)
        dec_output, _ = self.decoder(tgt, enc_output)

        # Classification
        classification_output = dec_output[:, -1, :]  # Take the final output representation
        classification_output = self.fc_out(classification_output)  # Apply a linear layer
        #classification_output = nn.functional.softmax(classification_output, dim=-1)  # Apply softmax

        # classification_output: (batch_size, num_classes)
        return classification_output

In [ ]:
PAD_IDX = 0
encoder = Encoder(INPUT_SIZE, HIDDEN_SIZE, N_LAYERS, N_HEADS, FF_SIZE, PAD_IDX, DROPOUT_RATE)
decoder = Decoder(INPUT_SIZE, HIDDEN_SIZE, N_LAYERS, N_HEADS, FF_SIZE, PAD_IDX, DROPOUT_RATE)

model = Transformer(encoder, decoder, OUTPUT_SIZE, PAD_IDX, HIDDEN_SIZE).to(device)

In [ ]:
def get_lr(step_num, warmup_steps=3000, d_model=512):
    step_num = max(1, step_num)
    arg1 = step_num ** -0.5
    arg2 = step_num * warmup_steps ** -1.5
    lrate = d_model ** -0.5 * min(arg1, arg2)
    return lrate


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=get_lr(0))

losses = []
accuracies = []


def calculate_accuracy(output, targets):
    predictions = output.argmax(dim=1, keepdim=True)  # assuming your output includes class scores
    correct = predictions.eq(targets.view_as(predictions)).sum().item()
    return correct / len(targets)


def train(early_stopping=True, patience=25):
    model.train() # Turn on the train mode
    step = 0
    losses = []
    accuracies = []
    val_losses = []
    val_accuracies = []
    best_val_loss = float('inf')
    best_model_state = None
    epochs_without_improvement = 0
    for epoch in range(1, N_EPOCHS + 1):
      total_loss = 0.0
      total_accuracy = 0.0
      for batch, i in enumerate(range(0, input.size(0) - 1, bptt)):
        data, targets = get_batch(input, torch.tensor(target_index), i, bptt)
        #print(data.size(), targets.size())
        #print(torch.isnan(data).any(), torch.isinf(data).any())
        #print(torch.isnan(targets).any(), torch.isinf(targets).any())

        optimizer.zero_grad()
        output = model(data, data)
        for name, param in model.named_parameters():
          if not torch.isfinite(param).all():
            print(f"Parameter {name} contains nan or inf.")
        if not torch.isfinite(output).all():
          print("Output contains nan or inf.")
        if not torch.isfinite(targets).all():
          print("Targets contain nan or inf.")
        #print("OPTSIZE: ", output.size())
        #print(output.view(-1, output.size(-1)).size(), targets.view(-1).size())
        loss = criterion(output.view(-1, output.size(-1)), targets.view(-1))
        #print(loss)
        loss.backward()
        w = torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)
        #print(w)

        for param_group in optimizer.param_groups:
            param_group['lr'] = get_lr(step + 1)

        optimizer.step()

        #print("STEP: ", step)
        step += 1

        total_loss += loss.item()
        total_accuracy += calculate_accuracy(output, targets)

      avg_loss = total_loss / len(range(0, input.size(0) - 1, bptt))
      avg_accuracy = total_accuracy / len(range(0, input.size(0) - 1, bptt))
      losses.append(avg_loss)
      accuracies.append(avg_accuracy)

      val_loss, val_accuracy = evaluate(model, val_input, val_target_index, criterion, bptt)
      val_losses.append(val_loss)
      val_accuracies.append(val_accuracy)

      print(f"Epoch {epoch}: Train Loss = {avg_loss}, Train Accuracy = {avg_accuracy}, Val Loss = {val_loss}, Val Accuracy = {val_accuracy}")

      # Early stopping
      if early_stopping:
          if val_loss < best_val_loss:
              best_val_loss = val_loss
              best_model_state = copy.deepcopy(model.state_dict())
              epochs_without_improvement = 0
          else:
              epochs_without_improvement += 1
              if epochs_without_improvement >= patience:
                  print(f"Early stopping after {epoch} epochs")
                  break

    if early_stopping:
        model.load_state_dict(best_model_state)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(losses, label='Training Loss')
    plt.title('Training Loss over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(accuracies, label='Training Accuracy')
    plt.title('Training Accuracy over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

    return model, losses, accuracies, val_losses, val_accuracies

# Testing and Evaluation (Model C)

In [ ]:
def evaluate(model, test_input, test_target_index, criterion, bptt):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    total_accuracy = 0.0

    with torch.no_grad():  # Disable gradient tracking
        for i in range(0, test_input.size(0) - 1, bptt):
            data, targets = get_batch(test_input, torch.tensor(test_target_index), i, bptt)
            output = model(data, data)
            loss = criterion(output.view(-1, output.size(-1)), targets.view(-1))
            total_loss += loss.item()
            total_accuracy += calculate_accuracy(output, targets)

    avg_loss = total_loss / len(range(0, test_input.size(0) - 1, bptt))
    avg_accuracy = total_accuracy / len(range(0, test_input.size(0) - 1, bptt))

    return avg_loss, avg_accuracy

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def perf_eval(model, test_input, test_target_index, criterion, bptt):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    total_accuracy = 0.0

    all_targets = []
    all_predictions = []

    with torch.no_grad():  # Disable gradient tracking
        for i in range(0, test_input.size(0) - 1, bptt):
            data, targets = get_batch(test_input, torch.tensor(test_target_index), i, bptt)
            output = model(data, data)
            print(output.size())
            loss = criterion(output.view(-1, output.size(-1)), targets.view(-1))
            total_loss += loss.item()
            total_accuracy += calculate_accuracy(output, targets)

            # Collect targets and predictions for metrics calculation
            all_targets.extend(targets.view(-1).cpu().numpy())
            _, predicted = torch.max(output, 1)
            all_predictions.extend(predicted.view(-1).cpu().numpy())

    avg_loss = total_loss / len(range(0, test_input.size(0) - 1, bptt))
    avg_accuracy = total_accuracy / len(range(0, test_input.size(0) - 1, bptt))

    # Calculate precision, recall, F1 score, and MCC with zero_division set to 0
    precision = precision_score(all_targets, all_predictions, average='weighted', zero_division=0)
    recall = recall_score(all_targets, all_predictions, average='weighted', zero_division=0)
    f1 = f1_score(all_targets, all_predictions, average='weighted', zero_division=0)

    return avg_loss, avg_accuracy, precision, recall, f1

In [ ]:
model, train_losses, train_accuracies, val_losses, val_accuracies = train()

test_loss, test_accuracy, precision, recall, f1 = perf_eval(model, test_input, test_target_index, criterion, bptt)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")